In [ ]:
import math

from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import ray

import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import numpy as np
import time

from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

import matplotlib.pyplot as plt

import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing

from typing import Union, Tuple
from torch_geometric.typing import OptTensor, OptPairTensor, Adj, Size

from torch import Tensor
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.conv import MessagePassing


from ROOT import TLorentzVector

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate

In [ ]:
data = torch.from_numpy( np.load('/data/github/data/diHiggs_kin_train_data.npy').astype(np.float32))
data2 = torch.from_numpy( np.load('/data/github/data/diHiggs_kin_test_data.npy').astype(np.float32))

In [ ]:
e1_, e2_ = [], []
for i in range(6):
    for j in range(6):
        if(i!=j):
            e1_.append(i)
            e2_.append(j)


def graph_trans(d, y=1, e1=e1_, e2=e2_):
    d = d/100.
    b1 = d[17], d[18], d[19], d[20]
    b2 = d[21], d[22], d[23], d[24]
    l1 = d[25], d[26], d[27], d[28]
    l2 = d[29], d[30], d[31], d[32]
    nu1 = d[33], d[34], d[35], d[36]
    nu2 = d[37], d[38], d[39], d[40]


    x = torch.tensor([b1, b2, l1, l2, nu1, nu2], dtype=torch.float)

    GNN_index = torch.tensor([e1, e2], dtype=torch.long)


    graph = Data(x=x, y=[y, y, y, y, y, y], GNN_index=GNN_index)
    return graph

In [ ]:
train_graph_list = []
for idx, d in enumerate(data):
    if(idx<=19000):
        y = 1
    else:
        y = 0
    train_graph_list.append(graph_trans(d, y))

test_graph_list = []
for idx, d in enumerate(data2):
    if(idx<=5000):
        y = 1
    else:
        y = 0
    test_graph_list.append(graph_trans(d, y))
    
train_loader = DataLoader(train_graph_list, 
                          batch_size=130,
                          shuffle=True)


test_loader = DataLoader(test_graph_list, batch_size=128, shuffle=False)

In [ ]:
class GNNConv(MessagePassing):
    def __init__(self, in_channels, hidden, out_channels):
        super().__init__(aggr='add')
        self.mlp1 = Seq(Linear(4*in_channels, hidden, bias=True),
                       nn.LeakyReLU())
        self.mlp2 = Seq(Linear(hidden+in_channels, out_channels, bias=True),
                       nn.LeakyReLU())


    def forward(self, x, GNN_index):
        out = self.propagate(GNN_index, x=x)
        out = torch.cat([x, out], dim=1)
        out = self.mlp2(out)

        return out

    def message(self, x_i, x_j):
        x_j = torch.cat([x_i, x_i*x_j, x_i+x_j, x_j-x_i], dim=1)
        x_j = self.mlp1(x_j)
        return x_j

In [ ]:
class EdgeConv(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp0 = Seq(Linear(4, 54, bias=True),
                               nn.LeakyReLU())
        self.conv1 = GNNConv(54, 231, 221)
        self.conv2 = GNNConv(221, 229, 164)
        self.conv3 = GNNConv(164, 131, 215)

        self.dropout = nn.Dropout(0.0)
        self.mlp1 = Seq(Linear(215*6, 54, bias=True),
                        nn.LeakyReLU(),
                        Linear(54, 2, bias=True),
                        nn.LeakyReLU())

    def forward(self, data):
        x, GNN_index = data.x, data.GNN_index

        x = self.mlp0(x)
        x = self.conv1(x, GNN_index)
        x = self.conv2(x, GNN_index)
        x = self.conv3(x, GNN_index)
        x = x.reshape(-1, x.shape[1]*6)
        x = self.mlp1(x)
        return F.softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = EdgeConv().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=9.203442233965491e-07,
                             betas=(0.9294469864692979, 0.9909760356430601),
                             weight_decay=0.03206031460677253,
                            )
scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma=1.0
)

In [ ]:
def train(model, train_loader):
    model.train()
    loss_val = []

    optimizer.zero_grad()
    for batch in train_loader:
        
        label = [1 if i==[1, 1, 1, 1, 1, 1] else 0 for i in batch.y]
        label = torch.tensor(label).to(device)
        out = model(batch.to(device))
    
        loss = criterion(out, label)
        
        loss.backward()
        optimizer.step()

        loss_val.append(loss.item())
        
    scheduler.step()
    
    loss = np.mean(loss_val)
    
    print(f'\n Train Epoch: {epoch} \tLoss: {loss:.6f}')
    
    return loss

In [ ]:
def test(model, test_loader):
    
    DNN_score = open('/data/github/result/EdgeConv_DNN_score.TXT', 'a')    

    model.eval()
    loss_val = []
    for idx, batch in enumerate(test_loader):
        out = model(batch.to(device)) 
        label = [1 if i==[1, 1, 1, 1, 1, 1] else 0 for i in batch.y]
        label = torch.tensor(label).to(device)
        loss = criterion(out, label)

        loss_val.append(loss.item())

        out = (out.cpu().detach().numpy())[:]
        if(idx==0):
            test_out = out[:]
            test_y = np.array(batch.y)[:]
        else:
            test_out = np.concatenate((test_out[:], out[:]), axis=0)
            test_y = np.concatenate((test_y[:], np.array(batch.y)[:]), axis=0)
    
    for n in range(0,len(test_out)):
        DNN_score.write(str((test_out[:,0:1][n].item())) + ' ' + str((test_out[:,1:2][n].item())) + ' ' + str((test_y[:,0:1][n].item())) + ' ' + str((test_y[:,1:2][n].item())) + ' ' + str((test_y[:,2:3][n].item())) + ' ' + str((test_y[:,3:4][n].item())) + ' ' + str((test_y[:,4:5][n].item())) + ' ' + str((test_y[:,5:6][n].item())))
        DNN_score.write('\n')

    loss = np.mean(loss_val)

    print(f'\nTest set: Loss: {loss:.6f}\n')
    print('----------------------------------------------------')

    return loss

In [ ]:
def Like(s, b, u, n):
    return math.e**((n*s+b)*math.log(u*s+b)-math.lgamma(n*s+b+1)-(u*s+b))

def IndiLikeRatioDis(NS1, NB1):
    return math.sqrt(-2*math.log((Like(NS1, NB1, 0.0, 1.0))/(Like(NS1, NB1, 1.0, 1.0))))

In [ ]:
DNN_score = open('/data/github/result/EdgeConv_DNN_score.TXT', 'w')    

Train_Loss = []
Test_Loss = []

epochs = 1
best_sig = 0

for epoch in range(1, epochs + 1):
    start=time.time()
    print('########### Training epoch {} start ###########'.format(epoch))
    
    train_loss = train(model, train_loader)
    test_loss = test(model, test_loader)

    Train_Loss.append(train_loss)
    Test_Loss.append(test_loss)
    
    epoch_ = epoch

    Results = np.loadtxt('/data/github/result/EdgeConv_DNN_score.TXT')

    Results = Results[25250*(epoch_-1):25250*(epoch_)]

    test_out = Results[:,:2]
    test_y = Results[:,2:]
    
    Results_hh_1=test_out[:,1:2][:5000]
    Results_tt_1=test_out[:,1:2][5000:5000+19000]
    Results_tw_1=test_out[:,1:2][5000+19000:5000+19000+800]
    Results_tth_1=test_out[:,1:2][5000+19000+800:5000+19000+800+100]
    Results_ttv_1=test_out[:,1:2][5000+19000+800+100:5000+19000+800+100+100]
    Results_llbj_1=test_out[:,1:2][5000+19000+800+100+100:5000+19000+800+100+100+240]
    Results_tatabb_1=test_out[:,1:2][5000+19000+800+100+100+240:5000+19000+800+100+100+240+10]

    plt.rc('text', usetex=True)
    plt.rc('font', family='Time New Roman')

    logs = False

    axislabels = [ r'$DNN $']

    Yaxislabels = [ r'$DNN $']

    Bmax = 1
    Bmin = 0
    plt.xlim(0, 1)
    bins = np.linspace(Bmin, Bmax,  25)
    plt.hist(Results_hh_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()

    ROC_Results = open('/data/github/result/EdgeConv_ROC'+str(epoch), 'w')

    test_sig = 0.0
    test_Nsig, test_Nbkg = 0, 0
    for temp_cut in np.linspace(0.9, 1.0, 10000):
        pre = (test_out[:, 1]-temp_cut>=0).astype(int)

        hh = sum(pre[:5000])
        tt = sum(pre[5000: 5000 + 19000])
        tw = sum(pre[5000 + 19000: 5000 + 19000 + 800])
        tth = sum(pre[5000 + 19000 + 800: 5000 + 19000 + 800 + 100])
        ttv = sum(pre[5000 + 19000 + 800 + 100: 5000 + 19000 + 800 + 100 + 100])
        llbj = sum(pre[5000 + 19000 + 800 + 100 + 100: 5000 + 19000 + 800 + 100 + 100 + 240])
        tatabb = sum(pre[5000 + 19000 + 800 + 100 + 100 + 240: 5000 + 19000 + 800 + 100 + 100 + 240 + 10])

        Nsig = round( float( 3000*0.0214964*hh/5000*(0.8**2)/(0.7**2) ), 3)
        Nbkg = round( float( 3000*(120.907*1.596*tt/19000 + 4.38354*tw/800 + 0.15258*1.27*tth/100 + 0.157968*1.54*ttv/100 + 1.22936*llbj/240 + 0.011392*tatabb/10)*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tt = round( float( 3000*120.907*1.596*tt/19000*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tw = round( float( 3000*4.38354*tw/800*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tth = round( float( 3000*0.15258*1.27*tth/100*(0.8**2)/(0.7**2) ), 3)
        Nbkg_ttv = round( float( 3000*0.157968*1.54*ttv/100*(0.8**2)/(0.7**2) ), 3)
        Nbkg_llbj = round( float( 3000*1.22936*llbj/240*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tatabb = round( float( 3000*0.011392*tatabb/10*(0.8**2)/(0.7**2) ), 3)
        SobSqrtB = round( float( IndiLikeRatioDis(float( Nsig ),float( Nbkg ) )  ) ,   3 )
        ROC_Results.write(str(Nsig) + ' ' + str(Nbkg) + ' ' + str(SobSqrtB) + ' ' + str(Nbkg_tt) + ' ' + str(Nbkg_tw) + ' ' + str(Nbkg_tth) + ' ' + str(Nbkg_ttv) + ' ' + str(Nbkg_llbj) + ' ' + str(Nbkg_tatabb)   ) 
        ROC_Results.write('\n')

    ROC_Results.close()    

    ROC_Results= np.loadtxt('/data/github/result/EdgeConv_ROC'+str(epoch))

    SB=[]
    hh=[]
    tt=[]
    tw=[]
    tth=[]
    ttv=[]
    llbj=[]
    tatabb=[]

    for n in range(len(ROC_Results)):  
        SB.append(ROC_Results[n][2])
        hh.append(ROC_Results[n][0])
        tt.append(ROC_Results[n][3])
        tw.append(ROC_Results[n][4])
        tth.append(ROC_Results[n][5])
        ttv.append(ROC_Results[n][6])
        llbj.append(ROC_Results[n][7])
        tatabb.append(ROC_Results[n][8])

    plt.plot(hh,SB, color='r', label='Significance')
    plt.xlabel(r'$ N_s $', fontsize=20)
    plt.ylabel(r'Significance', fontsize=20)
    plt.legend(loc='best', fontsize=15)
    plt.show()

    j=SB.index(max(SB))

    print('\nsignificance: {:.3f} hh: {:.3f} tt: {:.3f} tw: {:.3f} tth: {:.3f} ttv: {:.3f} llbj: {:.3f} tatabb: {:.3f} \n'.format(SB[j], hh[j], tt[j], tw[j], tth[j], ttv[j], llbj[j], tatabb[j]))
    
    if epoch % 1 == 0:
        sig = max(SB)
        best_sig = max(best_sig,sig)           

    end=time.time()

    print('* Best Significance : {:.3f} *'.format(best_sig))

    print('Epoch time: {:.2f} mins'.format((end-start)/60))
    print('='*69)      